In [ ]:
# ----------------- Import Required Modules ----------------- #
import tkinter as tk                  # Tkinter for GUI
from tkinter import ttk, messagebox   # ttk for themed widgets, messagebox for alerts
from PIL import Image, ImageTk        # PIL(Python Imaging Library) Pillow(used modern Python projects) for handling and resizing images
import re                             # re for regular expressions (validation)
import os                             # os for file path handling

# ----------------- Payroll Application Class ----------------- #
class PayrollApp(tk.Tk):  # Inherits from Tkinter main window
    def __init__(self):
        super().__init__()  # Initialize the Tk window
        self.title("Modern Payroll System")   # Window title
        self.geometry("700x750")              # Set window size
        self.config(bg="#F0F2F5")             # Set background color
        self.resizable(False, False)          # Disable window resizing

        # Load custom styles, icons, and create GUI widgets
        self._load_styles()
        self._load_icons()
        self._create_widgets_alt()

    # ----------------- Define Custom Styles ----------------- #
    def _load_styles(self):
        style = ttk.Style(self)
        style.theme_use('clam')  # Use clean modern 'clam' theme

        # Header label styling (big blue bar on top)
        style.configure('Header.TLabel', background="#3498DB", foreground="white",
                        font=("Segoe UI", 24, "bold"), padding=15)

        # General label styling
        style.configure('TLabel', background="white", foreground="#333333",
                        font=("Segoe UI", 11, "bold"), anchor="w")

        # Entry box (text field) styling
        style.configure('TEntry', fieldbackground="#ECF0F1", font=("Segoe UI", 11),
                        padding=10, relief="flat")

        # Change entry border when focused
        style.map('TEntry', fieldbackground=[('focus', '#E0F2F7')],
                  bordercolor=[('focus', '#3498DB')])

        # Button styling
        style.configure('TButton', background="#2ECC71", foreground="white",
                        font=("Segoe UI", 13, "bold"), padding=10, relief="flat")
        style.map('TButton', background=[('active', '#27AE60')], foreground=[('active', 'white')])

        # Label frame styling (group boxes)
        style.configure('TLabelframe', background="white", foreground="#333333",
                        font=("Segoe UI", 12, "bold"), borderwidth=1, relief="solid")
        style.configure('TLabelframe.Label', background="white", foreground="#333333",
                        font=("Segoe UI", 12, "bold"))

    # ----------------- Load Icons ----------------- #
    def _load_icons(self):
        """Loads small icons from 'icons' folder for UI decoration."""
        def load_icon(filename, size):
            path = os.path.join("icons", filename)
            if os.path.exists(path):
                try:
                    # Resize and load icon
                    return ImageTk.PhotoImage(Image.open(path).resize(size, Image.Resampling.LANCZOS))
                except Exception:
                    return None
            return None

        # Dictionary holding all icons
        self.icons = {
            "header": load_icon("payroll.png", (40, 40)),
            "name": load_icon("user.png", (25, 25)),
            "id": load_icon("id_card.png", (25, 25)),
            "salary": load_icon("salary.png", (25, 25)),
            "calc": load_icon("calculator.png", (28, 28))
        }

    # ----------------- Create the User Interface ----------------- #
    def _create_widgets_alt(self):
        # ---- Header Section ---- #
        header_frame = tk.Frame(self, bg="#3498DB", height=80)
        header_frame.pack(fill="x")

        header_label = ttk.Label(
            header_frame,
            text="Payroll Management System",
            image=self.icons["header"] if self.icons["header"] else None,
            compound='left',  # Show icon left of text
            style='Header.TLabel'
        )
        header_label.place(relx=0.5, rely=0.5, anchor="center")  # Center header text

        # ---- Employee Details Section ---- #
        input_frame = ttk.LabelFrame(self, text="Employee Details", style='TLabelframe', padding=20)
        input_frame.pack(padx=30, pady=20, fill="x")

        # String variables to store user input
        self.name_var = tk.StringVar()
        self.id_var = tk.StringVar()
        self.basic_var = tk.StringVar()

        # Name field
        ttk.Label(input_frame, text="Employee Name:",
                  image=self.icons["name"], compound='left').grid(row=0, column=0, sticky="w", pady=5)
        ttk.Entry(input_frame, textvariable=self.name_var, style='TEntry').grid(row=0, column=1, sticky="ew", pady=5)

        # ID field
        ttk.Label(input_frame, text="Employee ID:",
                  image=self.icons["id"], compound='left').grid(row=1, column=0, sticky="w", pady=5)
        ttk.Entry(input_frame, textvariable=self.id_var, style='TEntry').grid(row=1, column=1, sticky="ew", pady=5)

        # Salary field
        ttk.Label(input_frame, text="Basic Salary (₹):",
                  image=self.icons["salary"], compound='left').grid(row=2, column=0, sticky="w", pady=5)
        ttk.Entry(input_frame, textvariable=self.basic_var, style='TEntry').grid(row=2, column=1, sticky="ew", pady=5)

        # Make second column stretchable
        input_frame.columnconfigure(1, weight=1)

        # ---- Calculate Button ---- #
        calc_btn = ttk.Button(
            self, text="Calculate Salary",
            image=self.icons["calc"], compound='left',
            command=self._on_calculate, style='TButton'
        )
        calc_btn.pack(padx=30, pady=15, fill="x")

        # ---- Salary Slip Display ---- #
        result_frame = ttk.LabelFrame(self, text="Salary Slip", style='TLabelframe', padding=10)
        result_frame.pack(padx=30, pady=10, fill="both", expand=True)

        # Text box to display the salary slip
        self.result_box = tk.Text(result_frame, height=12, font=("Consolas", 11),
                                  bg="#F8F9FA", fg="#333333", bd=0, wrap="word", padx=10, pady=10)
        self.result_box.pack(side=tk.LEFT, fill="both", expand=True)

        # Scrollbar for the salary slip text box
        scroll = ttk.Scrollbar(result_frame, command=self.result_box.yview)
        scroll.pack(side=tk.RIGHT, fill="y")
        self.result_box['yscrollcommand'] = scroll.set
        self.result_box.config(state='disabled')  # Disable editing

    # ----------------- When Button is Clicked ----------------- #
    def _on_calculate(self):
        # Calculate salary and show result in text box
        slip = self._calculate_salary(self.name_var.get(), self.id_var.get(), self.basic_var.get())
        self.result_box.config(state='normal')  # Enable text editing temporarily
        self.result_box.delete(1.0, tk.END)     # Clear old result
        self.result_box.insert(tk.END, slip)    # Show new salary slip
        self.result_box.config(state='disabled')  # Disable editing again

    # ----------------- Salary Calculation Logic ----------------- #
    def _calculate_salary(self, name, emp_id, basic_str):
        try:
            # Clean and validate inputs
            name = name.strip() # strip - removes any extra spaces from the beginning and end of a string.
            emp_id = emp_id.strip()
            basic_str = basic_str.strip()

            # Validate employee name
            if not name:
                raise ValueError("Employee Name cannot be empty.")
            if not re.fullmatch(r"[A-Za-z\s]+", name):
                raise ValueError("Employee Name must contain only letters and spaces.")

            # Validate employee ID
            if not emp_id:
                raise ValueError("Employee ID cannot be empty.")

            # Validate basic salary
            if not basic_str or not basic_str.replace('.', '', 1).isdigit():
                raise ValueError("Basic Salary must be a valid number.")

            basic = float(basic_str)
            if basic < 0:
                raise ValueError("Basic Salary cannot be negative.")

            # Salary components calculation
            hra = 0.2 * basic     # HRA = 20% of basic
            da = 0.1 * basic      # DA = 10% of basic
            pf = 0.05 * basic     # PF = 5% of basic

            # Gross and net salary
            gross = basic + hra + da
            net = gross - pf

            # Create formatted salary slip text
            slip = f"""
📝 PAYROLL SUMMARY
----------------------------------
👤 Employee Name : {name}
🆔 Employee ID   : {emp_id}
----------------------------------
💰 Basic Pay     : ₹{basic:.2f}
🏠 HRA (20%)     : ₹{hra:.2f}
📈 DA (10%)      : ₹{da:.2f}
🏦 PF (5%)       : ₹{pf:.2f}
----------------------------------
📊 Gross Pay     : ₹{gross:.2f}
✅ Net Salary    : ₹{net:.2f}
"""
            return slip  # Return formatted slip text

        # Error handling for invalid inputs
        except ValueError as e:
            return f"Input Error: {e}"
        except Exception as e:
            return f"Unexpected error: {e}"

# ----------------- Run the App ----------------- #
if __name__ == "__main__":
    app = PayrollApp()  # Create app instance
    app.mainloop()      # Start Tkinter event loop
